In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle 
from scipy import  stats
import statsmodels.api as sm

def NationQtyToFile(item_id,store_id,preQty):
    if store_id == 0:
        store_id = 'all'
    preResultFile = open("TwoWeek79to93.csv","a")
    #preResultFile = open("SubmissionResult0507.csv","a")
    resultLine = [str(item_id),str(store_id),str(preQty)]
    printline = ",".join(resultLine)
    preResultFile.writelines(printline)
    preResultFile.write('\n')
    preResultFile.close()
    
def CorrPlot(dta):
    fig = plt.figure(figsize=(12,8))
    ax1=fig.add_subplot(211)
    fig = sm.graphics.tsa.plot_acf(dta,lags=40,ax=ax1)
    ax2 = fig.add_subplot(212)
    fig = sm.graphics.tsa.plot_pacf(dta,lags=40,ax=ax2)

def ARMApredict(QtyData):
        temp = []
        for i in range(len(QtyData )):
            if QtyData[i] >0:
                break;
        #temp.append(temp1[i])
        temp = QtyData[i:]
        temp =  np.array(temp)
        sortedTemp = sorted(temp,reverse = True)
        for i in range(3):
            temp[temp==sortedTemp[i]] = temp.mean()
        dta=pd.Series(temp)
        start_date = 1801
        end_date = start_date+len(temp)-1
        dta.index = pd.Index(sm.tsa.datetools.dates_from_range(str(start_date),str(end_date)))
        dta= dta.diff(1)[1:]
        try:
            arma_mod01 = sm.tsa.ARMA(dta,(5,1)).fit()
        except ValueError:
            arma_mod01 = sm.tsa.ARMA(dta,(3,1)).fit()
        except NameError:
            arma_mod01 = sm.tsa.ARMA(dta,(3,1)).fit()
        pre_start_date = end_date+1
        pre_end_date = end_date+13
        predicts = arma_mod01.predict(str(pre_start_date), str(pre_end_date), dynamic=True)
        qtyPrediction = QtyPredict(QtyData,predicts)
        return sum(qtyPrediction[-14:])

def QtyPredict(temp1,predicts ):
    for i in range(len(predicts)):
        temp1.append(temp1[-1]+predicts[i] )
    return temp1

f= open('itemCostDict.pkl','rb')
itemCostDict = pickle.load(f)
f.close()
f= open('QtyStoredateSortedData.pkl','rb')
QtyStoreDataDict = pickle.load(f)
f.close()
f= open('QtyNationdateSortedData.pkl','rb')
QtyNationDataDict = pickle.load(f)
f.close()

In [29]:
itemList = np.array(QtyStoreDataDict.keys())
for i in range(999,len(itemList)-1):
    item_id = itemList[i]
    print item_id
    print i
    for store_id in QtyStoreDataDict[item_id].keys():
        QtyData = QtyStoreDataDict[item_id][store_id]
        if sum(QtyData[-40:])>50:
            LastTwoWeeksQty = ARMApredict(QtyData)
            if LastTwoWeeksQty<0:
                LastTwoWeeksQty = 0
        else:
            LastTwoWeeksQty = np.mean(QtyData[-40:])*14
        NationQtyToFile(item_id,store_id,LastTwoWeeksQty)

for i in range(999,len(itemList)):
    item_id = itemList[i]
    print i
    print item_id
    QtyData = QtyNationDataDict[item_id]
    if sum(QtyData[-40:])>30:
        LastTwoWeeksQty = ARMApredict(QtyData)
        if LastTwoWeeksQty<0:
            LastTwoWeeksQty = 0
    else:
        LastTwoWeeksQty = np.mean(QtyData[-40:])*14
    NationQtyToFile(item_id,0,LastTwoWeeksQty)

999
26622


110


In [6]:
#================================
#===============================
QtyStoreTwoWeekDict = {}
for item_id in QtyStoreDataDict.keys():
    QtyStoreTwoWeekDict.setdefault(item_id,{})
    for store_id in QtyStoreDataDict[item_id].keys():
        QtyStoreTwoWeekDict [item_id][store_id] = []
        temp = QtyStoreDataDict[item_id][store_id]
        temp =np.array(temp)
        Qty = sum(temp[79:93])
        NationQtyToFile(item_id,store_id,Qty)
        
TwoWeekNationQtyDict = {}
for item_id in QtyNationDataDict.keys():
    TwoWeekNationQtyDict.setdefault(item_id,{})
    TwoWeekNationQtyDict[item_id] = []
    temp = QtyNationDataDict[item_id]
    temp =np.array(temp)
    Qty = sum(temp[79:93])
    NationQtyToFile(item_id,0,Qty)